### Starting point

In [1]:
# we prepare an empty directory for the demo
rm -Rf ~/metwork/layerapi2_demo
mkdir -p ~/metwork/layerapi2_demo
cd ~/metwork/layerapi2_demo

In [2]:
# we unset METWORK_LAYERS_PATH
unset METWORK_LAYERS_PATH

In [3]:
# just to have an international language for error messages
export LANG=C

In [4]:
# define two bash functions to be able to modify the current context
function layer_load() {
    eval "$(layer_load_bash_cmds --debug "$1")"
}

function layer_unload() {
    eval "$(layer_unload_bash_cmds --debug "$1")"
}

In [5]:
echo $METWORK_LAYERS_PATH

note: the environnement variable is not set

In [6]:
# let's see installed layers
layers

note: of course, no layer installed (because METWORK_LAYERS_PATH is empty)

### Bootstraping

In [7]:
# Let's bootstrap a first layer (label: layer1_label) in the "layer1" subdir
bootstrap_layer.sh layer1_label layer1

In [8]:
# Let's bootstrap a second layer (label: layer2_label) in the "layer2" subdir
bootstrap_layer.sh layer2_label layer2

In [9]:
find . -type d |grep layer

./layer1
./layer1/bin
./layer1/lib
./layer1/lib/python3.5
./layer1/lib/python3.5/site-packages
./layer1/lib/pkgconfig
./layer2
./layer2/bin
./layer2/lib
./layer2/lib/python3.5
./layer2/lib/python3.5/site-packages
./layer2/lib/pkgconfig


we have two empty layer structures

In [10]:
find . -type f |grep layer

./layer1/.layerapi2_label
./layer2/.layerapi2_label


In [11]:
cat layer1/.layerapi2_label

layer1_label


In [12]:
cat layer2/.layerapi2_label

layer2_label


note: label files are also bootstrapped

In [13]:
# let's say that layer2 depends on layer1
echo "layer1_label" >layer2/.layerapi2_dependencies

In [14]:
# let's see installed layers
layers

note: still nothing because METWORK_LAYERS_PATH is still empty

### Basic features

In [15]:
# Let's set a METWORK_LAYERS_PATH
export METWORK_LAYERS_PATH=/home/fab/metwork/layerapi2_demo

In [16]:
# let's see installed layers
layers

- layer1_label [/home/fab/metwork/layerapi2_demo/layer1]
- layer2_label [/home/fab/metwork/layerapi2_demo/layer2]


note : our two layers are found

In [17]:
layers --loaded-filter=yes

note: nothing because our two layers are not loaded

In [18]:
# same thing but with other utilities
is_layer_installed layer1_label
is_layer_loaded layer1_label

1
0


In [19]:
# let's add a tool in the layer1
cat >layer1/bin/mytool <<EOF
#!/bin/bash

echo "it works"
EOF
chmod +x layer1/bin/mytool

In [20]:
# let's test it manually
layer1/bin/mytool

it works


In [21]:
# let's test it through the PATH
mytool

bash: mytool: command not found


: 127

note: "not found" because mytool is not in the current PATH

In [22]:
# let's load the layer1
layer_load layer1_label

[DEBUG]: loading layer1_label[/home/fab/metwork/layerapi2_demo/layer1]


In [23]:
layers

- (*) layer1_label [/home/fab/metwork/layerapi2_demo/layer1]
- layer2_label [/home/fab/metwork/layerapi2_demo/layer2]


note: the layer1 is loaded

In [24]:
# let's test it through the PATH
mytool

it works


In [25]:
# let's unload the layer1
layer_unload layer1_label

[DEBUG]: unloading layer1_label[/home/fab/metwork/layerapi2_demo/layer1]


In [26]:
# let's test
mytool

bash: mytool: command not found


: 127

In [27]:
# if we want to execute mytool without changing the current context
layer_wrapper --layers=layer1_label -- mytool

it works


In [28]:
is_layer_loaded layer1_label

0


note: the current context is not changed, the layer is not loaded

### Dependencies

In [29]:
layers

- layer1_label [/home/fab/metwork/layerapi2_demo/layer1]
- layer2_label [/home/fab/metwork/layerapi2_demo/layer2]


note: nothing is loaded

In [30]:
cat layer2/.layerapi2_dependencies

layer1_label


note: the layer2 depends on layer1

In [31]:
# let's load the layer2 in the current context
layer_load layer2_label

[DEBUG]: layer layer2_label[/home/fab/metwork/layerapi2_demo/layer2] depends on not loaded layer layer1_label[/home/fab/metwork/layerapi2_demo/layer1] =>                             loading layer1_label
[DEBUG]: loading layer1_label[/home/fab/metwork/layerapi2_demo/layer1]
[DEBUG]: loading layer2_label[/home/fab/metwork/layerapi2_demo/layer2]


In [32]:
layers

- (*) layer1_label [/home/fab/metwork/layerapi2_demo/layer1]
- (*) layer2_label [/home/fab/metwork/layerapi2_demo/layer2]


note: both layers are loaded

In [33]:
# let's unload the layer1
layer_unload layer1_label

[DEBUG]: unloading layer1_label[/home/fab/metwork/layerapi2_demo/layer1]
[DEBUG]: layer layer2_label[/home/fab/metwork/layerapi2_demo/layer2] depends on missing dependency on layer layer1_label[/home/fab/metwork/layerapi2_demo/layer1]                                 => unloading layer2_label
[DEBUG]: unloading layer2_label[/home/fab/metwork/layerapi2_demo/layer2]


In [34]:
layers

- layer1_label [/home/fab/metwork/layerapi2_demo/layer1]
- layer2_label [/home/fab/metwork/layerapi2_demo/layer2]


note: the layer2 is also unloaded (because it depends on layer1)

### Conflicts
By default, python3@mfext is loaded. We want to use Python 2, so we will load python3@mfext


In [35]:
# let's load python2@mfext
layer_load python2@mfext

[DEBUG]: layer python2@mfext[/opt/metwork-mfext-master/opt/python2] depends on not loaded layer python2_core@mfext[/opt/metwork-mfext-master/opt/python2_core] =>                             loading python2_core@mfext
[DEBUG]: layer python2_core@mfext[/opt/metwork-mfext-master/opt/python2_core] conflicts with already loaded layer python3_core@mfext[/opt/metwork-mfext-master/opt/python3_core] =>                         unloading python3_core@mfext
[DEBUG]: unloading python3_core@mfext[/opt/metwork-mfext-master/opt/python3_core]
[DEBUG]: layer python3@mfext[/opt/metwork-mfext-master/opt/python3] depends on missing dependency on layer python3_core@mfext[/opt/metwork-mfext-master/opt/python3_core]                                 => unloading python3@mfext
[DEBUG]: unloading python3@mfext[/opt/metwork-mfext-master/opt/python3]
[DEBUG]: layer python3_devtools@mfext[/opt/metwork-mfext-master/opt/python3_devtools] depends on missing dependency on layer python3@mfext[/opt/metwork-mfext-master/op

Python 2 and its dependencies are loaded. Python 3 is in conflict wih Python 2. Python 3 and its dependencies are unloaded.